This notebook contains code to profile and clean the Historical DOB Permit Issuance data found here https://data.cityofnewyork.us/Housing-Development/Historical-DOB-Permit-Issuance/bty7-2jhb

# Download Dataset

In [56]:
import gzip
import os
import humanfriendly
import numpy as np
import pandas as pd

from openclean.data.source.socrata import Socrata


# Unique id for Historical DOB Permit Issuance data set
db_id = 'bty7-2jhb'
dataset = Socrata().dataset(db_id)

datafile = f'./{db_id}.tsv.gz'

# Download file if it doesn't exist
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as  f:
        print('Downloading ...\n')
        dataset.write(f)

fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print(f'Using "{dataset.name}" in file {datafile} of size {fsize}')


Using "Historical DOB Permit Issuance" in file ./bty7-2jhb.tsv.gz of size 321.34 MB


# Profiling the Data

We load the data in using opencleans stream method.

In [2]:
from openclean.pipeline import stream

ds_full = stream(datafile)


We used openclean profiles to create a summary of the stats for each column.

In [3]:
from openclean.profiling.column import DefaultColumnProfiler
# Example with first 20 columns
COLUMNS_SUBSET = ['BOROUGH','BIN','Number','Street','Job #',
                  'Job doc. #','Job Type','Self_Cert','Block',
                  'Lot','Community Board','Postcode','Bldg Type',
                  'Residential','Special District 1','Special District 2',
                  'Work Type','Permit Status','Filing Status','Permit Type'
]

ds = ds_full.select(columns=COLUMNS_SUBSET)

profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
BOROUGH,2428526,0,5,2.058862e-06,2.045161
BIN,2428526,0,300024,1.235416e-01,16.764061
Number,2428526,4,28639,1.179277e-02,11.933581
Street,2428526,4,20201,8.318228e-03,11.223448
Job #,2428526,0,1110544,4.572914e-01,19.723021
Job doc. #,2428526,0,12,4.941269e-06,0.496506
Job Type,2428526,0,6,2.470634e-06,1.855338
Self_Cert,2428526,1527841,1,1.110266e-06,0.000000
Block,2428526,498,13625,5.611550e-03,12.545550
Lot,2428526,507,1718,7.075727e-04,6.659702


We made a method to generate a histogram of a column in order to identify outliers.

In [4]:
def get_histogram(data_set, column_name):
    print(f'\n{column_name}:')
    value = data_set.distinct(column_name)
    for rank, val in enumerate(value.most_common()):
        v, freq = val
        print(f'{rank+1:<3} {v} {freq:>10,}')
# Example
get_histogram(ds_full, 'Street')


Street:
1   BROADWAY     68,853
2   PARK AVENUE     41,630
3   MADISON AVENUE     29,448
4   5 AVENUE     29,034
5   FIFTH AVENUE     20,375
6   3 AVENUE     16,342
7   LEXINGTON AVENUE     14,139
8   THIRD AVENUE     13,626
9   7 AVENUE     11,319
10  8 AVENUE     10,492
11  AVENUE OF THE AMERICAS     10,035
12  6 AVENUE      9,654
13  WEST END AVENUE      9,017
14  NORTHERN BOULEVARD      8,914
15  QUEENS BOULEVARD      8,845
16  HUDSON STREET      7,931
17  CENTRAL PARK WEST      7,447
18  RIVERSIDE DRIVE      7,385
19  FULTON STREET      7,341
20  JAMAICA AVENUE      7,302
21  AMSTERDAM AVENUE      6,939
22  2 AVENUE      6,793
23  WEST   57 STREET      6,756
24  ROOSEVELT AVENUE      6,702
25  1 AVENUE      6,309
26  GRAND STREET      6,196
27  SEVENTH AVENUE      6,047
28  ATLANTIC AVENUE      5,918
29  MAIN STREET      5,738
30  PARK AVENUE SOUTH      5,708
31  MYRTLE AVENUE      5,571
32  FLATBUSH AVENUE      5,472
33  FIRST AVENUE      5,466
34  WALL STREET      5,352
35  BED

# Cleaning the Data

### Cleaning Street Column

In [5]:
from openclean_geo.address.usstreet import StandardizeUSStreetName
from openclean.function.value.null import is_empty

def clean_street_name(name):
    # Replace empty data with 'N/A'
    if is_empty(name):
        return 'N/A'
    # Function to help standardize the street names
    street_func = StandardizeUSStreetName(characters='upper', alphanum=True, repeated=False)
    name = ''.join(street_func.apply([name], threads=None))
    # The conditional statements below are used to try and reduce the remaining
    # number of outlier data by fixing some common errors revealed in the histogram.
    if name == 'CLARKE PLACE EAST':
        name = 'EAST CLARKE PLACE'
    elif name == 'EAST BEDFORD PARK BLVD':
        name = 'BEDFORD PARK BLVD EAST'
    elif name == 'WTC':
        name = 'WORLD TRADE CTR'
    elif name == 'TIME SQ':
        name = 'TIMES SQ'
    elif name == 'PITT':
        name = 'PITT ST'
    elif name == 'BOGARDUS':
        name = 'BOGARDUS PLACE'
    elif name == 'NAGLE':
        name = 'NAGLE AVE'
    elif name == 'SHEPHERD':
        name = 'SHEPHERD AVE'

    split_name = name.split()
    if split_name[-1] in ['SSTREET', 'STRET', 'STREET', 'STREE']:
        split_name[-1] = 'ST'
    elif split_name[-1] == 'PL':
        split_name[-1] = 'PLACE'
    elif split_name[-1].isnumeric():
        split_name.append('ST')
    elif split_name[0] == 'ST':
        split_name[0] = 'SAINT'
    elif split_name[-1] == 'E':
        split_name[-1] = 'EAST'
    elif split_name[-1] == 'W':
        split_name[-1] = 'WEST'
    elif split_name[-1] == 'N':
        split_name[-1] = 'NORTH'
    elif split_name[-1] in ['S', 'SOUIH']:
        split_name[-1] = 'SOUTH'
    elif split_name[-1] in ['BLDV', 'BLV', 'BOULEVARD', 'BOOULEVARD']:
        split_name[-1] = 'BLVD'

    name = ' '.join(split_name)

    return name

def clean_street_data(ds):
    cleaned_data = ds.update('Street', lambda x: clean_street_name(x))
    return cleaned_data

# Street profile after cleaning
ds_full = clean_street_data(ds_full)
street_data = ds_full.select(columns=['Street'])
after_clean_street = street_data.profile(default_profiler=DefaultColumnProfiler)
# The number of unique values in Street is significantly lower after
# standardizing and cleaning the data
after_clean_street.stats()

,total,empty,distinct,uniqueness,entropy
Street,2428526,0,8576,0.003531,10.576541


### Cleaning Number Column

In [6]:
def clean_number(num):
    if is_empty(num):
        return 'N/A'
    # remove any leading zero's
    num = num.lstrip('0')
    if len(num) == 0:
        return 'N/A'

    return num

def clean_number_data(ds):
    cleaned_data = ds.update('Number', lambda x: clean_number(x)) 
    return cleaned_data

ds_full = clean_number_data(ds_full)

### Clean Permit Type Column

Profiling revealed that there is only one row missing a value for Permit Type. The Work Type for that row is PL, which stands for Plumbing. Every other tuple in the dataset that had a Work Type of PL also had a Permit Type PL. So I think it's fair to assume that the missing Permit Type should be filled with PL since it has Work Type PL.

In [7]:
# Replace the only missing Permit Type with PL since its work type is PL and
# all other rows with work type PL have permit type PL also
def clean_permit_type(ds):
    cleaned_data = ds.update('Permit Type', lambda x: 'PL' if is_empty(x) else x) 
    return cleaned_data

ds_full = clean_permit_type(ds_full)

### Clean Block and Lot Column

Block and Lot are both values that are assigned by the Deparment of Finance and depend on the address of the building. This web application https://stevemorse.org/vital/nycblocklot.html takes in Borough, Number, and Street and outputs the Block and Lot values by scraping http://maps.nyc.gov/doitt/nycitymap/ . So I used this web app to try and fill in some of the missing Block and Lot data.

In [8]:
import requests
import urllib.parse

# Make a request the url to try and find the block and lot for an address.
# Replace missing data with N/A if it fails to find a value
# This requires you to first clean street and number data
def get_block_and_lot(borough, number, street, block, lot):
    # Return if block and lot are already filled in
    if not is_empty(block) and not is_empty(lot):
        return borough, number, street, block, lot
    if not is_empty(number) and not is_empty(street):
        req = f'https://stevemorse.org/vital/nycblocklot.php?borough={borough.title()}&number={number}&street={urllib.parse.quote(street.title())}'
        r = requests.get(req)
        if r.text == "Callback('?', '?');":
            # Try again without applying title() to street value
            req = f'https://stevemorse.org/vital/nycblocklot.php?borough={borough.title()}&number={number}&street={urllib.parse.quote(street)}'
            r = requests.get(req)
        if r.status_code == 200:
            r_str = r.text.removeprefix('Callback(').removesuffix(');').replace("'", '')
            block_lot = r_str.split(',')
            if is_empty(block):
                block = block_lot[0].strip()
            if is_empty(lot):
                lot = block_lot[1].strip()
    # Return 'N/A' if the web app was unable to find the block and lot data
    # for this input
    block = 'N/A' if block == '?' else block
    lot = 'N/A' if lot == '?' else lot
    return borough, number, street, block, lot

def clean_block_and_lot(ds):
    needed_columns = ['BOROUGH','Number','Street','Block','Lot']
    cleaned_data = ds.update(needed_columns, lambda bo, n, s, bl, l: get_block_and_lot(bo, n, s, bl, l))
    return cleaned_data

ds_full = clean_block_and_lot(ds_full)

### Clean Community Board Data

Community Board is a 3-digit identifier with the first digit being the Borough code and the last two digits representing the community board code for that building. This data contained a lot of missing and incorrect values for community board. We were unable to find an API to identify the correct community board code for an address, so we are just replacing all empty and invalid data with 'N/A'. 

In [9]:
# Replace missing or incorrect data with N/A
def fix_community_board_data(data):
    is_valid = True
    if not is_empty(data) and len(data) == 3:
        for i in range(3):
            if not data[i].isnumeric():
                is_valid = False
                break
    else:
        is_valid = False
    if is_valid and data[0].isnumeric():
        if int(data[0]) > 5:
            is_valid = False
    if is_valid:
        return data
    else:
        return 'N/A'

def clean_community_board_data(ds):
    cleaned_data = ds.update('Community Board', lambda x: fix_community_board_data(x)) 
    return cleaned_data

ds_full = clean_community_board_data(ds_full)

### Clean Remaining missing data

For columns that are marked as not required or don't have a means to identify the correct value for missing data, we simply replaced the empty value with the string 'N/A'. 

In [10]:
# Replace unrequired and missing data with N/A
def replace_empty_data(self_cert, postcode, bldg_type, residential, special1, special2, work_type, permit_status):
    if is_empty(self_cert):
        self_cert = 'N/A'
    if is_empty(postcode):
        postcode = 'N/A'
    if is_empty(bldg_type):
        bldg_type = 'N/A'
    if is_empty(residential):
        residential = 'N/A'
    if is_empty(special1):
        special1 = 'N/A'
    if is_empty(special2):
        special2 = 'N/A'
    if is_empty(work_type):
        work_type = 'N/A'
    if is_empty(permit_status):
        permit_status = 'N/A'
    return self_cert, postcode, bldg_type, residential, special1, special2, work_type, permit_status

def clean_missing_data(ds):
    COLUMNS= [
        'Self_Cert',
        'Postcode',
        'Bldg Type',
        'Residential',
        'Special District 1',
        'Special District 2',
        'Work Type',
        'Permit Status'
    ]
    cleaned_data = ds.update(COLUMNS, lambda s, po, b, r, s1, s2, w, pe: replace_empty_data(s, po, b, r, s1, s2, w, pe))
    return cleaned_data

ds_full = clean_missing_data(ds_full)

## Profiling after cleaning the data

In [11]:
ds = ds_full.select(columns=COLUMNS_SUBSET)

profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
BOROUGH,2428526,0,5,2.058862e-06,2.045161
BIN,2428526,0,300024,1.235416e-01,16.764061
Number,2428526,0,28600,1.177669e-02,11.932774
Street,2428526,0,8576,3.531360e-03,10.576541
Job #,2428526,0,1110544,4.572914e-01,19.723021
Job doc. #,2428526,0,12,4.941269e-06,0.496506
Job Type,2428526,0,6,2.470634e-06,1.855338
Self_Cert,2428526,0,2,8.235448e-07,0.951343
Block,2428526,0,13626,5.610811e-03,12.545786
Lot,2428526,0,1719,7.078368e-04,6.661165


## Cols 21 - 40

In [1]:
# load
data = pd.read_csv('Historical_DOB_Permit_Issuance.csv')
cols = data[data.columns[20:40]]
cols.columns

Index(['Permit Sequence #', 'Permit Subtype', 'Oil Gas', 'Site Fill',
       'Filing Date', 'Issuance Date', 'Expiration Date', 'Job Start Date',
       'Permittee's First Name', 'Permittee's Last Name',
       'Permittee's Business Name', 'Permittee's Phone #',
       'Permittee's License Type', 'Permittee's License #',
       'Act as Superintendent', 'Permittee's Other Title', 'HIC License',
       'Site Safety Mgr's First Name', 'Site Safety Mgr's Last Name',
       'Site Safety Mgr Business Name'],
      dtype='object')

In [58]:
cols.head()

,Permit Sequence #,Permit Subtype,Oil Gas,Site Fill,Filing Date,Issuance Date,Expiration Date,Job Start Date,Permittee's First Name,Permittee's Last Name,Permittee's Business Name,Permittee's Phone #,Permittee's License Type,Permittee's License #,Act as Superintendent,Permittee's Other Title,HIC License,Site Safety Mgr's First Name,Site Safety Mgr's Last Name,Site Safety Mgr Business Name
0,1,NaN,NaN,NaN,2010-11-05T00:00:00,2010-11-05T00:00:00,2011-11-05T00:00:00,2010-11-05T00:00:00,LAWRENCE,LEVINE,"PAR PLUMBING CO., INC",2129261088,MASTER PLUMBER,161,NaN,NaN,NaN,NaN,NaN,NaN
1,12,FN,NaN,NONE,2012-01-30T00:00:00,2012-01-30T00:00:00,2013-01-29T00:00:00,2002-08-08T00:00:00,ANTHONY,RASULO,RIVERBAY CORP,7183203300,GENERAL CONTRACTOR,1962,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NONE,2008-02-04T00:00:00,2008-02-04T00:00:00,2009-02-03T00:00:00,2005-08-29T00:00:00,OSCAR,JACKSON,PERFECT PLUMBING & HETING CORP,7185157055,MASTER PLUMBER,594,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NONE,1998-08-31T00:00:00,1998-08-31T00:00:00,1999-08-31T00:00:00,1998-08-31T00:00:00,GERI,KAUUMBA,GOWIE PLUMBING,7188821281,MASTER PLUMBER,1137,Y,NaN,NaN,NaN,NaN,NaN
4,1,MH,NaN,NONE,2007-04-30T00:00:00,2007-04-30T00:00:00,2008-01-08T00:00:00,2007-04-30T00:00:00,GARY,ZYSMAN,THE DU-RITE INC,2013877000,GENERAL CONTRACTOR,9872,Y,NaN,NaN,NaN,NaN,NaN


In [59]:
cols.tail()

,Permit Sequence #,Permit Subtype,Oil Gas,Site Fill,Filing Date,Issuance Date,Expiration Date,Job Start Date,Permittee's First Name,Permittee's Last Name,Permittee's Business Name,Permittee's Phone #,Permittee's License Type,Permittee's License #,Act as Superintendent,Permittee's Other Title,HIC License,Site Safety Mgr's First Name,Site Safety Mgr's Last Name,Site Safety Mgr Business Name
2428521,1,NaN,NaN,ON-SITE,2003-10-08T00:00:00,2003-10-08T00:00:00,2004-04-24T00:00:00,2003-10-08T00:00:00,SALVATORE,CALCAGNO,SALVATORE CALCAGNO CONSTRUCTION,7.18443e+09,GENERAL CONTRACTOR,3107,Y,NaN,NaN,NaN,NaN,NaN
2428522,1,NaN,NaN,NONE,1996-07-29T00:00:00,1997-07-28T00:00:00,1998-07-10T00:00:00,1996-07-29T00:00:00,VITO,MELELEO,LAVINIO CONSTR. INC.,7.18949e+09,GENERAL CONTRACTOR,5209,Y,GC,NaN,NaN,NaN,NaN
2428523,1,NaN,NaN,NONE,1999-07-09T00:00:00,1999-07-09T00:00:00,2000-07-08T00:00:00,1999-07-09T00:00:00,CHARLES,BERWIND,A. BERWIND PLUMBING & HEATING,5.16594e+09,MASTER PLUMBER,180,Y,NaN,NaN,NaN,NaN,NaN
2428524,2,NaN,NaN,NONE,1996-06-25T00:00:00,1996-06-25T00:00:00,1997-06-07T00:00:00,1994-08-09T00:00:00,FRANK,CIOLLO,FRANK C. PL. & HEATING CORP.,7.18981e+09,MASTER PLUMBER,1357,Y,NaN,NaN,NaN,NaN,NaN
2428525,1,NaN,NaN,OFF-SITE,1999-09-20T00:00:00,1999-09-20T00:00:00,2000-01-01T00:00:00,1999-09-20T00:00:00,KENNETH,FROHLICK,GATEWAY DEMOLITION CORP.,7.18359e+09,GENERAL CONTRACTOR,1023,Y,NaN,NaN,NaN,NaN,NaN


In [61]:
# types of cols with missing values
cols.dtypes[cols.isnull().any()]

Permit Subtype                   object
Oil Gas                          object
Site Fill                        object
Expiration Date                  object
Job Start Date                   object
Permittee's First Name           object
Permittee's Last Name            object
Permittee's Business Name        object
Permittee's Phone #              object
Permittee's License Type         object
Permittee's License #            object
Act as Superintendent            object
Permittee's Other Title          object
HIC License                      object
Site Safety Mgr's First Name     object
Site Safety Mgr's Last Name      object
Site Safety Mgr Business Name    object
dtype: object

In [62]:
# show the number (percentage) of missing values for each col
x = cols.isnull().sum()
y = cols.isnull().sum() / cols.shape[0] * 100
z = {'Number of missing values' : x, 'Percentage of missing values' : y}
df = pd.DataFrame(z, columns = ['Number of missing values', 'Percentage of missing values'])
df.sort_values(by = 'Percentage of missing values', ascending = False)

,Number of missing values,Percentage of missing values
Site Safety Mgr Business Name,2420665,99.676306
Site Safety Mgr's First Name,2418289,99.578469
Site Safety Mgr's Last Name,2418265,99.577480
HIC License,2401722,98.896285
Oil Gas,2397492,98.722106
Permittee's Other Title,2160344,88.957005
Permit Subtype,1010293,41.601078
Act as Superintendent,833300,34.312995
Permittee's License Type,254952,10.498220
Permittee's License #,221198,9.108323


## Data Clean Cols 21 - 40

- Permit Subtype

In [10]:
cols['Permit Subtype'].value_counts()

OT    585308
FN    215908
MH    214293
SH    125409
SP     80185
BL     51996
FP     50161
FB     31488
SF     31106
FS     17035
SD      9189
EA      5005
CH       909
FA       239
SC         2
Name: Permit Subtype, dtype: int64

In [11]:
cols['Permit Subtype'].isnull().sum()

1010293

In [63]:
# Fill missing value with NA
cols['Permit Subtype'].fillna("N/A", inplace=True)

In [64]:
cols['Permit Subtype'].isnull().sum()

0

- Oil Gas

In [65]:
cols['Oil Gas'].value_counts()

OIL    29215
GAS     1819
Name: Oil Gas, dtype: int64

In [66]:
cols['Oil Gas'].isnull().sum()

2397492

In [67]:
# Fill missing value with NA
cols['Oil Gas'].fillna("N/A", inplace=True)

- Site Fill

In [68]:
cols["Site Fill"].value_counts()

NONE                   1553658
NOT APPLICABLE          371778
ON-SITE                 243255
OFF-SITE                 83449
USE UNDER 300 CU.YD       8683
Name: Site Fill, dtype: int64

In [69]:
cols['Site Fill'].fillna("N/A", inplace=True)
cols['Site Fill'].replace("NONE", "N/A", inplace=True)
cols['Site Fill'].replace("USE UNDER 300 CU.YD", "N/A", inplace=True)

In [70]:
cols["Site Fill"].isnull().sum()

0

- Filling Data

In [2]:
cols['Filing Date'].value_counts()

2007-03-29T00:00:00    998
2007-03-30T00:00:00    981
2006-12-28T00:00:00    927
2008-01-07T00:00:00    920
2007-12-28T00:00:00    900
                      ... 
2007-05-28T00:00:00      1
2012-10-21T00:00:00      1
1989-05-24T00:00:00      1
2008-05-18T00:00:00      1
2010-05-02T00:00:00      1
Name: Filing Date, Length: 6415, dtype: int64

In [3]:
# Remove Time after Date
cols['Filing Date'] = cols['Filing Date'].str.split("T")
cols['Filing Date'] = cols['Filing Date'].apply(lambda x: x[0])

cols['Filing Date'].value_counts()

2007-03-29    998
2007-03-30    981
2006-12-28    927
2008-01-07    920
2007-12-28    900
             ... 
2013-06-25      1
2009-01-11      1
2010-07-10      1
2012-05-20      1
2009-04-04      1
Name: Filing Date, Length: 6415, dtype: int64

In [4]:
#filter out invalid date

startDate = '1989-01-01'
endDate = '2013-12-31'

afterStartDate = cols['Job Start Date'] >= startDate
beforeEndDate = cols['Job Start Date'] <= endDate
daysBetween = afterStartDate & beforeEndDate

cols['Filing Date'] = cols['Filing Date'].loc[daysBetween]

cols['Filing Date'].value_counts()

2007-03-29    998
2007-03-30    981
2006-12-28    927
2008-01-07    920
2007-12-28    900
             ... 
2010-06-12      1
2011-02-13      1
2005-08-06      1
2011-12-26      1
2008-05-26      1
Name: Filing Date, Length: 6415, dtype: int64

- Issuance Date

In [5]:
cols['Issuance Date'].value_counts()

2007-03-29T00:00:00    994
2007-03-30T00:00:00    959
2006-12-28T00:00:00    947
2007-12-28T00:00:00    918
2008-06-27T00:00:00    909
                      ... 
2008-04-13T00:00:00      1
2010-11-14T00:00:00      1
2011-11-27T00:00:00      1
2013-04-13T00:00:00      1
2013-01-01T00:00:00      1
Name: Issuance Date, Length: 6409, dtype: int64

In [6]:
cols['Issuance Date'].isnull().sum()

0

In [7]:
# Remove Time after Date
cols['Issuance Date'] = cols['Issuance Date'].str.split("T")
cols['Issuance Date'] = cols['Issuance Date'].apply(lambda x: x[0])

cols['Issuance Date'].value_counts()

2007-03-29    994
2007-03-30    959
2006-12-28    947
2007-12-28    918
2008-06-27    909
             ... 
2012-02-25      1
2008-05-18      1
2013-01-12      1
2011-07-31      1
1989-05-16      1
Name: Issuance Date, Length: 6409, dtype: int64

In [8]:
#filter out invalid date

startDate = '1989-01-01'
endDate = '2013-12-31'

afterStartDate = cols['Job Start Date'] >= startDate
beforeEndDate = cols['Job Start Date'] <= endDate
daysBetween = afterStartDate & beforeEndDate

cols['Issuance Date'] = cols['Issuance Date'].loc[daysBetween]

cols['Issuance Date'].value_counts()

2007-03-29    994
2007-03-30    959
2006-12-28    947
2007-12-28    918
2008-06-27    909
             ... 
2012-01-22      1
2005-08-07      1
2008-01-01      1
2006-06-18      1
1989-05-16      1
Name: Issuance Date, Length: 6409, dtype: int64

- Expiration Date

In [9]:
cols['Expiration Date'].value_counts()

2007-12-31T00:00:00    18638
2006-12-31T00:00:00    18065
2005-12-31T00:00:00    16359
2004-12-31T00:00:00    13974
2009-04-01T00:00:00    11426
                       ...  
1989-08-30T00:00:00        1
1990-04-03T00:00:00        1
1990-01-15T00:00:00        1
2015-02-05T00:00:00        1
1989-11-25T00:00:00        1
Name: Expiration Date, Length: 9024, dtype: int64

In [10]:
cols['Expiration Date'].isnull().sum()

4

In [11]:
# take care of missing value
cols['Expiration Date'].fillna("N/A", inplace=True)

In [12]:
# Remove Time after Date
cols['Expiration Date'] = cols['Expiration Date'].str.split("T")
cols['Expiration Date'] = cols['Expiration Date'].apply(lambda x: x[0])

cols['Expiration Date'].value_counts()

2007-12-31    18638
2006-12-31    18065
2005-12-31    16359
2004-12-31    13974
2009-04-01    11426
              ...  
1989-10-22        1
1989-09-25        1
2014-06-24        1
1989-05-11        1
2015-05-05        1
Name: Expiration Date, Length: 9025, dtype: int64

In [13]:
#filter out invalid date

startDate = '1989-01-01'
endDate = '2013-12-31'

afterStartDate = cols['Job Start Date'] >= startDate
beforeEndDate = cols['Job Start Date'] <= endDate
daysBetween = afterStartDate & beforeEndDate

cols['Expiration Date'] = cols['Expiration Date'].loc[daysBetween]

cols['Expiration Date'].value_counts()

2007-12-31    18634
2006-12-31    18063
2005-12-31    16357
2004-12-31    13972
2009-04-01    11426
              ...  
1989-10-24        1
1989-06-05        1
1989-09-19        1
1989-11-17        1
1989-10-05        1
Name: Expiration Date, Length: 9011, dtype: int64

- Job Start Date

In [14]:
cols['Job Start Date'].value_counts()

2008-06-27T00:00:00    1376
2008-06-25T00:00:00    1095
2007-07-17T00:00:00    1077
2004-06-09T00:00:00    1063
2008-07-29T00:00:00    1051
                       ... 
1996-06-09T00:00:00       1
2008-04-20T00:00:00       1
2007-01-28T00:00:00       1
2016-06-16T00:00:00       1
2205-02-22T00:00:00       1
Name: Job Start Date, Length: 7475, dtype: int64

In [15]:
cols['Job Start Date'].isnull().sum()

3

In [16]:
cols['Job Start Date'].fillna("N/A", inplace=True)

In [17]:
cols['Job Start Date'] = cols['Job Start Date'].str.split("T")
cols['Job Start Date'] = cols['Job Start Date'].apply(lambda x: x[0])

cols['Job Start Date'].value_counts()

2008-06-27    1376
2008-06-25    1095
2007-07-17    1077
2004-06-09    1063
2008-07-29    1051
              ... 
2213-04-08       1
1992-10-24       1
1996-05-12       1
2010-03-06       1
1993-12-04       1
Name: Job Start Date, Length: 7476, dtype: int64

In [18]:
#filter out invalid date

startDate = '1989-01-01'
endDate = '2013-12-31'

afterStartDate = cols['Job Start Date'] >= startDate
beforeEndDate = cols['Job Start Date'] <= endDate
daysBetween = afterStartDate & beforeEndDate

cols['Job Start Date'] = cols['Job Start Date'].loc[daysBetween]
cols['Job Start Date'].value_counts()

2008-06-27    1376
2008-06-25    1095
2007-07-17    1077
2004-06-09    1063
2008-07-29    1051
              ... 
2009-01-03       1
1989-08-19       1
2009-08-29       1
2008-07-20       1
2010-11-27       1
Name: Job Start Date, Length: 7261, dtype: int64

- Permittee's First Name

In [19]:
cols["Permittee's First Name"].isnull().sum()

71

In [20]:
# issues: having different symbols before or after
# ex: _ (__)` MR.  \{ . /ANN`   -   (, JR)   X.
# contains first name and middle name (or middle initial)
cols["Permittee's First Name"].value_counts()

JOHN             112451
MICHAEL           76081
ROBERT            69481
JOSEPH            57833
PETER             52864
                  ...  
VENKETACHALAM         1
ZORY                  1
ANDUL                 1
QING YUN              1
DELROSE               1
Name: Permittee's First Name, Length: 35865, dtype: int64

In [21]:
# Getting the first name only
# some names contain '-', " ", "_", "/" between first and middle name
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.split(" ").str[0]
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.split("-").str[0]
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.split("/").str[0]
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.split("\\").str[0]
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.split("_").str[0]

# further clean the first name
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("_")
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("`")
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("MR.")
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("\\")
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("{")
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip(".")                                                                      
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip("-")                                                                    
cols["Permittee's First Name"] = cols["Permittee's First Name"].str.strip(",")

# Not-a-name & missing value rows are given "NA"
cols["Permittee's First Name"].str.isnumeric().replace("N/A",inplace=True)
cols["Permittee's First Name"].fillna("N/A", inplace=True)

In [22]:
cols["Permittee's First Name"].value_counts()

JOHN         112921
ICHAEL        76390
OBERT         69635
JOSEPH        58011
PETE          53722
              ...  
ZAME              1
ARLILYN           1
EVERHEART         1
ZONA              1
ARICELA           1
Name: Permittee's First Name, Length: 28985, dtype: int64

In [23]:
cols["Permittee's First Name"].isnull().sum()

0

- Permittee's Last Name

In [24]:
# having similar issue with "Permittee's First Name"
cols["Permittee's Last Name"].value_counts()

SINGH          28500
WHITE          19733
LEE            16399
MARTINEZ       15557
BROWN          14565
               ...  
ABIUS              1
KAYUM              1
UHIARAMONTE        1
STAINER            1
MIRONEAUB          1
Name: Permittee's Last Name, Length: 85623, dtype: int64

In [25]:
cols["Permittee's Last Name"].isnull().sum()

88

In [26]:
# some names contain '-', " ", "_", "/" between last name & other coarse strings
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.split(" ").str[0]
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.split("-").str[0]
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.split("/").str[0]
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.split("\\").str[0]
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.split("_").str[0]

# further clean the Last name
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip("_")
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip("`")
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip("\\")
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip("{")
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip(".")                                                                      
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip("-")                                                                    
cols["Permittee's Last Name"] = cols["Permittee's Last Name"].str.strip(",")

# Not-a-name & missing value rows are given "NA"
cols["Permittee's Last Name"].str.isnumeric().replace("N/A",inplace=True)
cols["Permittee's Last Name"].fillna("N/A", inplace=True)

In [27]:
cols["Permittee's Last Name"].value_counts()

SINGH        28530
WHITE        19817
LEE          16425
MARTINEZ     15560
BROWN        14578
             ...  
BOGDANOW         1
SAKICHAND        1
LEAU             1
ANAVIAN          1
WIGDEN           1
Name: Permittee's Last Name, Length: 80925, dtype: int64

In [28]:
cols["Permittee's Last Name"].isnull().sum()

0

- Permittee's Business Name

In [29]:
cols["Permittee's Business Name"].value_counts()

ROCKLEDGE SCAFFOLD             7732
STRUCTURE TONE INC             7461
EVEREST SCAFFOLDING INC        7368
OWNER                          5090
THE PLUMBING COMPANY INC       4739
                               ... 
PILLA CONSTRUCTION                1
U.S.A.SIGNS OF  AMERICA INC       1
NATCO PLG & HTG INC               1
RICHARD PLBG & HTG                1
S.Z. CONSTRUCTION CO.             1
Name: Permittee's Business Name, Length: 351176, dtype: int64

In [30]:
cols["Permittee's Business Name"].isnull().sum()

31650

In [31]:
# Missing value means the name is not available (replace with NA)
#cols["Permittee's Business Name"].fillna("NA")
cols["Permittee's Business Name"].replace(("N.A", "na","n.a"), ("N/A","N/A","N/A"),inplace=True)

# some contains business address instead of business name
streetValue = ["st", "St", "ST", "street", "Street", "STREET"]
cols["Permittee's Business Name"].str.contains('|'.join(streetValue)).replace("N/A", inplace=True)

In [32]:
cols["Permittee's Business Name"].value_counts()

ROCKLEDGE SCAFFOLD                 7732
STRUCTURE TONE INC                 7461
EVEREST SCAFFOLDING INC            7368
OWNER                              5090
THE PLUMBING COMPANY INC           4739
                                   ... 
S. SINGH CONSTRUCTION                 1
MICHAEL CATSIMALIS PLBG & HTG.        1
SCHUMACHER & FARLEY P & H CORP.       1
PJ  MECH.                             1
J 7 M CONTRACTORS CORP                1
Name: Permittee's Business Name, Length: 351176, dtype: int64

- Permittee's Phone #

In [33]:
cols["Permittee's Phone #"].isnull().sum()

304

In [34]:
# some #s followed by a .0 (xxxxxxxxx.0)
# missing value means no # is available
cols["Permittee's Phone #"] = cols["Permittee's Phone #"].astype(str)
cols["Permittee's Phone #"] = cols["Permittee's Phone #"].str.split(".")
cols["Permittee's Phone #"] = cols["Permittee's Phone #"].apply(lambda x: x[0])
cols["Permittee's Phone #"].fillna("N/A",inplace=True)

In [35]:
cols["Permittee's Phone #"].isnull().sum()

0

In [36]:
cols["Permittee's Phone #"].value_counts()

2124816100    16556
7183924921    11749
9144230400    11396
2129261088     9812
7185894900     9110
              ...  
7186675960        1
7183584450        1
7182072169        1
5162970229        1
5164485540        1
Name: Permittee's Phone #, Length: 145122, dtype: int64

- Permittee's License Type

In [37]:
cols["Permittee's License Type"].value_counts()

GENERAL CONTRACTOR             1343651
MASTER PLUMBER                  528867
FIRE SUPPRESSION CONTRACTOR     128567
OIL BURNER INSTALLER             58574
SIGN HANGER                      50004
OWNER                            36295
NO WORK                          15367
DEMOLITION CONTRACTOR             5188
PROFESSIONAL ENGINEER             3433
REGISTERED ARCHITECT              2944
HOME IMPROVEMENT CONTRACTOR        684
Name: Permittee's License Type, dtype: int64

In [38]:
cols["Permittee's License Type"].isnull().sum()

254952

In [39]:
# Not sure what "NO WORK" means, but can't do anything to it
# missing value means no # is available
cols["Permittee's License Type"].fillna("N/A",inplace=True)

In [40]:
cols["Permittee's License Type"].isnull().sum()

0

- Permittee's License #

In [41]:
cols["Permittee's License #"].value_counts()

0        77664
2660     14497
1982     13270
2593     11559
626       9391
         ...  
37532        1
24366        1
25359        1
18713        1
10252        1
Name: Permittee's License #, Length: 49833, dtype: int64

In [42]:
cols["Permittee's License #"].isnull().sum()

221198

In [43]:
# a single digit from 0 to 9 means not available, or wrong input

singleNumber = ['0','1','2','3','4','5','6','7','8','9']
cols["Permittee's License #"].replace(singleNumber,'N/A',inplace=True)


In [44]:
# missing value means no # is available

cols["Permittee's License #"].str.strip(" ")
cols["Permittee's License #"].str.strip("\\")
cols["Permittee's License #"].str.strip(",")
cols["Permittee's License #"].str.strip("`")
cols["Permittee's License #"].str.strip("*")

cols["Permittee's License #"] = cols["Permittee's License #"].astype(str)
cols["Permittee's License #"] = cols["Permittee's License #"].str.split(".")
cols["Permittee's License #"] = cols["Permittee's License #"].apply(lambda x: x[0])
cols["Permittee's License #"].fillna("N/A",inplace=True)

In [45]:
cols["Permittee's License #"].isnull().sum()

0

- Act as Superintendent

In [46]:
cols['Act as Superintendent'].value_counts()

Y    1586974
N       8252
Name: Act as Superintendent, dtype: int64

In [47]:
cols['Act as Superintendent'].isnull().sum()

833300

In [48]:
# Fill missing value with NA
cols['Act as Superintendent'].fillna("N/A", inplace=True)

In [49]:
cols['Act as Superintendent'].isnull().sum()

0

- Permittee's Other Title

In [50]:
cols["Permittee's Other Title"].value_counts()

GC                105964
G.C.               63698
G.C                15668
CONTRACTOR         13935
G/C                 9598
                   ...  
FS PIPING CONT         1
MSTR FIRE SUPA         1
F.S. CON               1
MASTER ELEC.           1
PLUMBING CONTR         1
Name: Permittee's Other Title, Length: 2795, dtype: int64

In [51]:
cols["Permittee's Other Title"].isnull().sum()

2160344

In [52]:
GC = ["GC","GC .","G.C .","GCC.","G?C","G.,C.","G..C.","G,C,","G.C. (SIGNS)","DEMO CON","DEMO. CONTRACTR","GEN CONTACTOR","G .C.","G.CONTRACTOR","GEN  CONTRACTOR","G.C.","_GC","_G.C","G.C","G C","GC.","G.C..","G/C","_G.C.","___GC","G. C.","G.G..","G. C","G..C","G..","G/C/","G,C.","GEN. CONTRACTOR","GEN CONTRACTOR","GEN CONTR.","GEN. CONTR.","GENERAL CONTR.","GENERAL CONTRAC","GC/OWNER","GEN CON","GENERAL","GEN CONT"]
CONTRACTOR = ["CONTR.","CONTR","CONT","CON.","CONT.","C"]
DEMO_CONTRACTOR = ["DEMO CONTR","DEMO. CONTR.","DEMO.CON.","DEMO CONT","DEMO CONTR{","DEMO CONT.","DEMO.CON","DEMO. CONT.","DEMO. CONTRACTO","DEMO CONTR.","DEMO.","DEMO.CONTRACTOR","DEMOL CONTR"]
OIL_BURN_INST = ["OIL BURNER INS.","OIL BURNER INSL","OIL BURNER INST","B INSTALLER","OILBURNER","O.B.INSTALLER","FUEL OIL INST","OIL BURNER INS","B. INSTALLER","OIL BURNER INST","OIL BURNER","FUEL OIL","OIL BUR. INSTR.","OBI","OIL INSTALLER","OIL INSTALLER","OIL BURN. INSTA","OIL BUR.","OIL BRN INSTL","TANK INSTALLER","FUEL OIL INST.","BOILER INSTALL.","BOILER INSTALL","BOILER INSTALLE","BURNER INSTALLE","O.B. INSTALLER","O.B.I.","OB INSTALLER","O.B.","BOIL INSTALLER",""]
SIGN = ["SIGN MFG.","SIGN MFG","SIGN MGF","SIGN HANGER-164","SIGN HGR","SIGN MGR","SIGN HANG. #168","SIGN MANUFACT.","SIGN RIGGER","SIGN ERECTOR","SGN.HNGR.","SIGN HANGER-168","SIGN HANGERS","SIGN BROKER","SIGN HANGER-157","SG MFG","SGN HNGR","SIGN CONTRACTOR","SIGN MFGR.",]

cols["Permittee's Other Title"].replace(GC,"GENERAL CONTRACTOR",inplace=True)
cols["Permittee's Other Title"].replace(CONTRACTOR,"CONTRACTOR",inplace=True)
cols["Permittee's Other Title"].replace(DEMO_CONTRACTOR,"DEMOLITION CONTRACTOR",inplace=True)
cols["Permittee's Other Title"].replace(OIL_BURN_INST,"OIL BURNER INSTALLER",inplace=True)
cols["Permittee's Other Title"].replace(SIGN,"SIGN HANGER",inplace=True)

- HIC License

In [53]:
cols["HIC License"].value_counts()

0.0          694
9999999.0    545
45141.0      364
817339.0     325
1251318.0    295
            ... 
897900.0       1
897889.0       1
897886.0       1
897864.0       1
887263.0       1
Name: HIC License, Length: 6076, dtype: int64

In [54]:
cols["HIC License"].isnull().sum()

2401722

In [55]:
# remove ending .0s
cols["HIC License"] = cols["HIC License"].astype(str)
cols["HIC License"] = cols["HIC License"].str.split(".")
cols["HIC License"] = cols["HIC License"].apply(lambda x: x[0])
singleNumber = ['0','1','2','3','4','5','6','7','8','9']

# remove invalid inputs
cols["HIC License"].replace(singleNumber,'N/A',inplace=True)
cols["HIC License"].replace('nan','N/A',inplace=True)

# Fill missing value with NA
cols["HIC License"].fillna("N/A", inplace=True)

- Site Safety Mgr's First Name

In [56]:
cols["Site Safety Mgr's First Name"].value_counts()

JOHN              503
MICHAEL           476
ADMINISTRATIVE    376
JOSEPH            299
FRANK             294
                 ... 
CONRAD              1
JEAN                1
JONATHAN            1
HENDERSON           1
KENNETGH            1
Name: Site Safety Mgr's First Name, Length: 506, dtype: int64

In [57]:
cols["Site Safety Mgr's First Name"].isnull().sum()

2418289

In [58]:
# some names contain '-', " ", "_", "/" between name & other coarse strings
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.split(" ").str[0]
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.split("-").str[0]
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.split("/").str[0]
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.split("\\").str[0]
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.split("_").str[0]

# further clean the name
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip("_")
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip("`")
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip("\\")
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip("{")
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip(".")                                                                      
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip("-")                                                                    
cols["Site Safety Mgr's First Name"] = cols["Site Safety Mgr's First Name"].str.strip(",")

# Not-a-name & missing value rows are given "NA"
cols["Site Safety Mgr's First Name"].str.isnumeric().replace("N/A",inplace=True)
cols["Site Safety Mgr's First Name"].fillna("N/A", inplace=True)

- Site Safety Mgr's Last Name

In [59]:
cols["Site Safety Mgr's Last Name"].value_counts()

WAIVER       376
ESPOSITO      96
SAME          94
ALBUNIO       85
STUTO         82
            ... 
SENZON         1
CUMMINGS       1
DULIN          1
ALLEGRINI      1
SCHUEEN        1
Name: Site Safety Mgr's Last Name, Length: 1235, dtype: int64

In [60]:
cols["Site Safety Mgr's Last Name"].isnull().sum()

2418265

In [61]:
# some names contain '-', " ", "_", "/" between name & other coarse strings
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.split(" ").str[0]
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.split("-").str[0]
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.split("/").str[0]
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.split("\\").str[0]
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.split("_").str[0]

# further clean the name
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip("_")
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip("`")
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip("\\")
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip("{")
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip(".")                                                                      
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip("-")                                                                    
cols["Site Safety Mgr's Last Name"] = cols["Site Safety Mgr's Last Name"].str.strip(",")

# Not-a-name & missing value rows are given "NA"
cols["Site Safety Mgr's Last Name"].str.isnumeric().replace("N/A",inplace=True)
cols["Site Safety Mgr's Last Name"].fillna("N/A", inplace=True)

- Site Safety Mgr Business Name

In [62]:
cols["Site Safety Mgr Business Name"].value_counts()

TOTAL SAFETY CONSULTING          507
SITE SAFETY LLC                  428
NYC DEPT. OF BUILDINGS           378
BOVIS LEND LEASE                 272
SITE SAFETY, LLC                 178
                                ... 
ARWINGTON CONST.                   1
JACK GREEN ASSOCIATES              1
RO-SAL                             1
EBERHART BROTHERS INC.             1
ABCO PEERLESS SPRINKLER CORP.      1
Name: Site Safety Mgr Business Name, Length: 1257, dtype: int64

In [63]:
cols["Site Safety Mgr Business Name"].isnull().sum()

2420665

In [64]:
# clean the name
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip("_")
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip("`")
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip("\\")
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip("{")
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip(".")                                                                      
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip("-")                                                                    
cols["Site Safety Mgr Business Name"] = cols["Site Safety Mgr Business Name"].str.strip(",")


# Not-a-name & missing value rows are given "NA"
cols["Site Safety Mgr Business Name"].str.isnumeric().replace("N/A",inplace=True)
cols["Site Safety Mgr Business Name"].fillna("N/A", inplace=True)

## Export Result Data Set Columns 21-40

In [ ]:
cols.to_csv(r'/Users/weichen/Desktop/resultCols.csv', index = False)